# Lab 02: Optical Character Recognition (OCR) with Azure AI Vision

## Overview
Optical Character Recognition (OCR) enables you to extract text from images. Azure AI Vision's Read API uses advanced AI models to read printed and handwritten text from images and documents. In this lab, you'll:
- Extract text from images using the Read API
- Process and display detected text with line and word boundaries
- Visualize text detection results with bounding boxes

## Prerequisites
- An Azure subscription with an Azure AI Vision resource
- The endpoint and key for your Azure AI Vision resource

## Learning Objectives
- Use the Read API for text extraction from images
- Process OCR results including text blocks, lines, and words
- Visualize detected text with bounding polygons
- Handle different image types and text orientations

## Step 1: Install Required Packages

In [ ]:
# Install required packages
# !pip install python-dotenv azure-ai-vision-imageanalysis==1.0.0 matplotlib pillow

## Step 2: Import Required Libraries

In [ ]:
from dotenv import load_dotenv
import os
import time
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

%matplotlib inline

print("✓ Libraries imported successfully")

## Step 3: Configure Azure AI Vision Credentials

In [ ]:
# Load configuration from .env file
load_dotenv('python/read-text/.env')

ai_endpoint = os.getenv('AI_SERVICE_ENDPOINT')
ai_key = os.getenv('AI_SERVICE_KEY')

# Validate credentials
if not ai_endpoint or not ai_key or 'your_' in ai_endpoint or 'your_' in ai_key:
    print("⚠ Warning: Please configure your Azure AI Vision credentials")
else:
    print(f"✓ Endpoint configured: {ai_endpoint[:30]}...")
    print("✓ API key loaded")

## Step 4: Authenticate Azure AI Vision Client

In [ ]:
# Create authenticated client
cv_client = ImageAnalysisClient(
    endpoint=ai_endpoint,
    credential=AzureKeyCredential(ai_key)
)

print("✓ Azure AI Vision client authenticated successfully")

## Step 5: Read Text from an Image

Let's use the Read API to extract text from an image. We'll use an image containing the Gettysburg Address.

In [ ]:
# Specify the image to read
image_file = 'python/read-text/images/Lincoln.jpg'

# Display the image
print(f"Reading text from: {image_file}")
img = Image.open(image_file)
plt.figure(figsize=(12, 10))
plt.imshow(img)
plt.axis('off')
plt.title('Image with Text to Extract')
plt.show()

print(f"Image size: {img.size[0]}x{img.size[1]} pixels")

### 5.1 Extract Text Using Read API

In [ ]:
# Read the image data
with open(image_file, "rb") as f:
    image_data = f.read()

# Call the Read API
print("\nExtracting text from image...")
result = cv_client.analyze(
    image_data=image_data,
    visual_features=[VisualFeatures.READ]
)

print("✓ Text extraction complete")

### 5.2 Display Extracted Text

In [ ]:
# Display the extracted text
if result.read is not None:
    print("\n=== Extracted Text ===")
    print("="*60)
    
    # Iterate through blocks of text
    for block in result.read.blocks:
        for line in block.lines:
            print(line.text)
    
    print("="*60)
    
    # Display statistics
    total_lines = sum(len(block.lines) for block in result.read.blocks)
    total_words = sum(len(line.words) for block in result.read.blocks for line in block.lines)
    
    print(f"\nStatistics:")
    print(f"  Blocks: {len(result.read.blocks)}")
    print(f"  Lines: {total_lines}")
    print(f"  Words: {total_words}")
else:
    print("No text detected in the image")

## Step 6: Visualize Text Detection

Let's visualize the detected text by drawing bounding boxes around lines and words.

### 6.1 Annotate Lines of Text

In [ ]:
def annotate_lines(image_file, detected_text):
    """
    Draw bounding boxes around detected lines of text.
    
    Args:
        image_file: Path to the image file
        detected_text: Read result from Azure AI Vision
    """
    print('\nAnnotating lines of text in image...')
    
    # Prepare image for drawing
    image = Image.open(image_file)
    fig = plt.figure(figsize=(image.width/100, image.height/100))
    plt.axis('off')
    draw = ImageDraw.Draw(image)
    color = 'cyan'
    
    for block in detected_text.blocks:
        for line in block.lines:
            # Draw line bounding polygon
            r = line.bounding_polygon
            polygon = [(r[0].x, r[0].y), (r[1].x, r[1].y), 
                      (r[2].x, r[2].y), (r[3].x, r[3].y)]
            draw.polygon(polygon, outline=color, width=3)
    
    # Display annotated image
    plt.imshow(image)
    plt.tight_layout(pad=0)
    plt.title('Detected Lines of Text')
    plt.show()
    print('✓ Line annotation complete')

# Annotate lines in the image
if result.read is not None:
    annotate_lines(image_file, result.read)

### 6.2 Annotate Individual Words

In [ ]:
def annotate_words(image_file, detected_text):
    """
    Draw bounding boxes around individual words.
    
    Args:
        image_file: Path to the image file
        detected_text: Read result from Azure AI Vision
    """
    print('\nAnnotating individual words in image...')
    
    # Prepare image for drawing
    image = Image.open(image_file)
    fig = plt.figure(figsize=(image.width/100, image.height/100))
    plt.axis('off')
    draw = ImageDraw.Draw(image)
    color = 'magenta'
    
    for block in detected_text.blocks:
        for line in block.lines:
            for word in line.words:
                # Draw word bounding polygon
                r = word.bounding_polygon
                polygon = [(r[0].x, r[0].y), (r[1].x, r[1].y), 
                          (r[2].x, r[2].y), (r[3].x, r[3].y)]
                draw.polygon(polygon, outline=color, width=2)
    
    # Display annotated image
    plt.imshow(image)
    plt.tight_layout(pad=0)
    plt.title('Detected Words')
    plt.show()
    print('✓ Word annotation complete')

# Annotate words in the image
if result.read is not None:
    annotate_words(image_file, result.read)

## Step 7: Detailed Text Analysis

Let's examine the detected text in more detail, including confidence scores and bounding box coordinates.

In [ ]:
if result.read is not None:
    print("\n=== Detailed Text Analysis ===")
    
    for block_idx, block in enumerate(result.read.blocks, 1):
        print(f"\nBlock {block_idx}:")
        
        for line_idx, line in enumerate(block.lines, 1):
            print(f"\n  Line {line_idx}: \"{line.text}\"")
            
            # Show bounding polygon
            polygon_points = [(p.x, p.y) for p in line.bounding_polygon]
            print(f"    Bounding polygon: {polygon_points}")
            
            # Show individual words
            print(f"    Words ({len(line.words)}):")
            for word in line.words:
                print(f"      • {word.text} (confidence: {word.confidence:.2%})")

## Step 8: Try Different Images

Let's process different types of images with text.

In [ ]:
# List available images
import glob

image_dir = 'python/read-text/images'
available_images = glob.glob(f"{image_dir}/*.jpg")

print("Available images for OCR:")
for idx, img_path in enumerate(available_images, 1):
    print(f"{idx}. {os.path.basename(img_path)}")

In [ ]:
def read_and_display_text(image_path):
    """
    Read text from an image and display results.
    
    Args:
        image_path: Path to the image file
    """
    print(f"\n{'='*60}")
    print(f"Processing: {os.path.basename(image_path)}")
    print('='*60)
    
    # Display original image
    img = Image.open(image_path)
    plt.figure(figsize=(10, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Original: {os.path.basename(image_path)}')
    plt.show()
    
    # Read text
    with open(image_path, "rb") as f:
        image_data = f.read()
    
    result = cv_client.analyze(
        image_data=image_data,
        visual_features=[VisualFeatures.READ]
    )
    
    # Display extracted text
    if result.read is not None:
        print("\nExtracted Text:")
        print("-" * 60)
        for block in result.read.blocks:
            for line in block.lines:
                print(line.text)
        print("-" * 60)
        
        # Show annotated image
        annotate_lines(image_path, result.read)
    else:
        print("No text detected")

# Uncomment to process additional images
# for image_path in available_images:
#     read_and_display_text(image_path)

## Step 9: Extract Text from a URL

You can also analyze images from URLs without downloading them first.

In [ ]:
# Example: Reading text from an image URL
# Note: Replace with your own image URL containing text

# image_url = "https://example.com/image-with-text.jpg"

# result = cv_client.analyze_from_url(
#     image_url=image_url,
#     visual_features=[VisualFeatures.READ]
# )

# if result.read is not None:
#     print("Text from URL:")
#     for block in result.read.blocks:
#         for line in block.lines:
#             print(line.text)

print("To use URL-based analysis, uncomment the code above and provide a valid image URL")

## Summary

In this lab, you learned how to:
- ✓ Use Azure AI Vision's Read API for text extraction
- ✓ Process OCR results including blocks, lines, and words
- ✓ Visualize detected text with bounding polygons
- ✓ Extract detailed information including confidence scores
- ✓ Process multiple images programmatically

## Key Concepts

- **Blocks**: Groups of text lines that are logically related
- **Lines**: Horizontal sequences of words
- **Words**: Individual text elements with bounding polygons
- **Bounding Polygon**: Four-point polygon defining the text location
- **Confidence Score**: AI model's confidence in the recognized text (0-1)

## Next Steps

- Try the **Advanced OCR** notebook to explore:
  - Handwritten text recognition
  - Multi-language text detection
  - Document layout analysis
  - Text extraction from complex documents
- Explore [Azure AI Vision Read API documentation](https://learn.microsoft.com/azure/ai-services/computer-vision/overview-ocr)

## Clean Up

Remember to manage your Azure resources appropriately to avoid unexpected charges.